In [1]:
import pandas as pd
from IPython.display import Markdown
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = (
    pd
    .read_csv('../data/external/data.csv')
).sample(5000)
df = df.assign(
    site = df.site.str.split("/").apply(lambda x: f"{x[0]}//{x[2]}")
)
display(Markdown("### Conjunto de dados"))
display(df.head())

df_dict = pd.read_csv('../data/external/dicionario.csv')
display(Markdown("### Dicionário de dados"))
display(df_dict.head())

### Conjunto de dados

,text,datetime,category,label,site
6122,Retrospectiva 2017: relembre os fatos que marc...,30/12/2017 12h43,politica,true,https://g1.globo.com
5998,Cenário: Exercícios aéreos e controle de front...,23 de agosto de 2017,politica,true,http://internacional.estadao.com.br
1854,"Safadeza! Antes de deixar o poder, governo Di...",14/06/2016,politica,fake,https://www.diariodobrasil.org
1383,"Empréstimos secretos do governo petista: ""Bras...",30/08/2016,politica,fake,https://www.diariodobrasil.org
4423,"A categoria “Endurance” tem boas disputas, ca...",22 de março de 2018,ciencia_tecnologia,true,http://esportes.estadao.com.br


### Dicionário de dados

,variavel,significado,tipo,valores
0,text,"Coluna contendo o texto das matérias, sendo ca...",text,string
1,datetime,Coluna contendo a data de publicação das matérias,time,datetime
2,category,Coluna contetndo a categoria na qual cada maté...,nominal,"['politica', 'tv_celebridades', 'sociedade_cot..."
3,label,Coluna contendo os rótulos se a notícia é fals...,nominal,"['fake', 'true']"
4,site,Coluna contendo o link do site em que a notíci...,nominal,string


In [3]:
target_column = "label"
nominal_columns = (
    df_dict
    .query(
        "tipo == 'nominal' and variavel != @target_column"
    )
    .variavel
    .to_list()
)
text_columns = (
    df_dict
    .query(
        "tipo == 'text' and variavel != @target_column"
    )
    .variavel
    .to_list()
)
time_columns = (
    df_dict
    .query(
        "tipo == 'time' and variavel != @target_column"
    )
    .variavel
    .to_list()
)

In [4]:
nominal_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ("missing", SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ("encoder", OneHotEncoder(sparse=False, handle_unknown='ignore')), # Codificação de variáveis
    ("selector", VarianceThreshold(threshold=.1)), # Seleção de variáveis
    ("normalizer", StandardScaler()), # Normalização
])


text_preprocessor = Pipeline([
    ("bow", CountVectorizer(max_features=2000))
    # ("selector", VarianceThreshold(threshold=.1)), # Seleção de variáveis
    # ("normalizer", StandardScaler()), # Normalização
])


time_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    # ("missing", SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)), # Codificação de variáveis
    # ("selector", VarianceThreshold(threshold=.1)), # Seleção de variáveis
    # ("normalizer", StandardScaler()), # Normalização
])

In [5]:
text_columns

['text']

In [6]:
preprocessor = ColumnTransformer([
    ("nominal", nominal_preprocessor, nominal_columns),
    ("text", text_preprocessor, "text"),
    ("time", time_preprocessor, time_columns)
])
model = LinearSVC()
approach = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model)
])

In [7]:
X = df.drop(columns=target_column, axis=1)
y = df[[target_column]]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [8]:
approach.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('nominal',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('selector',
                                                                   VarianceThreshold(threshold=0.1)),
                                                                  ('normalizer',
                                                                   StandardScaler())]),
                                                  ['category', 'site']),
                                                 ('text',
                                                  Pipeline(steps=[('bow',
                                                                   CountVectorizer(max_features=2000))]),
                                                  'text'),
                                                 ('time',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['datetime'])])),
                ('model', LinearSVC())])

In [9]:
y_test_hat = approach.predict(X_test)

In [10]:
accuracy_score(y_test, y_test_hat)

0.974